In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from forex_rnn import ohlc_rnn

In [ ]:
class forex_env:
    sample = 10
    num = 0
    count = 0
    forex = []
        
    def get_next_set(self):
        self.num = np.random.randint(0, 790)
        print("        " + str(self.num))
        #if self.num == 56:
        #    self.num = self.num + 1
        #elif self.num == 330:
        #    self.num = self.num + 1
        #elif self.num == 331:
          #  self.num = self.num + 1
        #elif self.num == 180:
            #self.num = self.num + 1
        self.day = 0
        self.forex = pd.read_csv("ForexData/" + str(self.num),encoding ='utf-16le')
        if len(self.forex.columns) < 2:
            self.forex = pd.read_csv("ForexData/" + str(self.num))
        if self.forex.count(axis=1)[0] == 6:
            self.forex.columns = ['Tid', 'Dealable', 'Pair', 'DateTime', 'Buy', 'Sell']
            if self.forex['Sell'].dtype is not np.dtype(np.float64):
                #print(self.forex['Sell'].dtype)
                self.forex.columns = ['Tid', 'Pair', 'DateTime', 'Buy', 'Sell', 'Dealable']
            del self.forex['Tid']
            del self.forex['Dealable']
            del self.forex['Pair']
        else:
            self.forex.columns = ['Tid', 'Pair', 'DateTime', 'Buy', 'Sell']
            del self.forex['Tid']
            del self.forex['Pair']
        
        self.forex.DateTime = pd.to_datetime(self.forex.DateTime)
        self.forex = self.forex.set_index('DateTime')
        self.forex = self.forex.sort_index()

        #ohlc = self.forex.resample(self.sample).ohlc()
        self.num = self.num + 1
    
    def get_next_day(self):
        self.get_next_set()
        #print(self.day)
        ts = np.random.randint(0,5)
        r = np.random.randint(0, 4)
        day_forex = self.forex[pd.Timestamp(self.forex.iloc[0].name)+pd.DateOffset(hours=24)*ts:
                               pd.Timestamp(self.forex.iloc[0].name)+pd.DateOffset(hours=24)*(ts+1)]
        if r is 0:
            day_forex = day_forex.between_time("4:00PM", "7:00PM")
        elif r is 1:
            day_forex = day_forex.between_time("7:00PM", "1:00AM")
        elif r is 2:
            day_forex = day_forex.between_time("2:00AM", "7:00AM")
        elif r is 3:
            day_forex = day_forex.between_time("7:00AM", "4:00PM")
        self.day = self.day + 1
        return day_forex
        
    def check_if_done(self):
        if self.day < 700:
            return True
        return False
    
    def get_d_forex(self):
        return d_forex(self)

    def reset(self):
        self.num = 0
        self.day = 0
        self.get_next_set()


In [ ]:
class d_forex:
    sample = 10
    forex = []
    ohlc = []
    index = 0
    o_i = 0
    done = True
    def __init__(self, env, time=10):
        self.env = env
        self.sample = 10
        self.initialize()
    def get_next_datum(self):
        try:
            ret = self.forex.iloc[self.index]
            self.index = self.index+1
            if ret.name > self.ohlc.iloc[self.o_i+1].name:
                self.o_i = self.o_i+1
        except:
            self.done=False
            return []
        return ret
    def get_next_ohlc(self):
        return self.ohlc.iloc[0:-2], self.ohlc.iloc[1:-1]
    def step(self):
        datum = self.get_next_datum()
        if len(datum) == 0:
            self.done = False
        return datum
    def initialize(self):
        self.forex = self.env.get_next_day()
        self.index = 0
        self.o_i = 0
        time = str(self.sample)+'T'
        self.ohlc = self.forex.resample(time).ohlc()
        self.ohlc = self.ohlc.dropna(axis=0, how='any')
        if self.ohlc.empty:
            self.env.get_next_set()
            self.initialize()
        self.done = True
    def check_if_done(self):
        return self.done

In [ ]:
class stock:
    def __init__(self):
        #self.rnn = ohlc_rnn(tf.train.AdamOptimizer(1e-5))
        self.env = forex_env()
        self.day = self.env.get_d_forex()
        start = self.day.step()
        self.st = [start]
        g = tf.Graph()
        with g.as_default():
            self.ohlc = ohlc_rnn(tf.train.AdamOptimizer())
            self.sess = tf.Session(graph=g)
            save = tf.train.Saver()
            save.restore(self.sess, "graph/model.ckpt")
        self.init_ohlc = self.ohlc.get_init_state(self.sess)
        self.state = self.init_ohlc
        self.choice = [0,0,1]
        self.ch_price = 0
        self.reward = 0
        self.settle = 0
        #print(self.st)
    def get_first_ohlc(self):
        tmp = self.day.step()
        while pd.Timestamp(tmp.name) < pd.Timestamp(self.st[0].name)+pd.DateOffset(minutes=9, seconds=(60-pd.Timestamp(self.st[0].name).second)):
            #print(tmp)
            self.st.append(tmp)
            tmp = self.day.step()
        df = pd.DataFrame(self.st)
        self.oh = df.resample("10T").ohlc()
        self.oh, self.state = self.ohlc.run_rnn(self.oh, self.sess,self.state)
    def next_data(self):
        tmp = self.day.step()
        #print(self.day.check_if_done())
        #print(tmp)
        if self.day.check_if_done():
            if not pd.Timestamp(tmp.name) < pd.Timestamp(self.st[0].name)+pd.DateOffset(minutes=19, seconds=(60-pd.Timestamp(self.st[0].name).second)):
                df = pd.DataFrame(self.st)
                self.oh = df.resample("10T").ohlc()
                self.oh, self.state = self.ohlc.run_rnn(self.oh, self.sess,self.state)
                self.st = [tmp]
            else:
                self.st.append(tmp)
            return [self.choice[0], self.choice[1], self.choice[2], self.ch_price, tmp['Buy'], tmp['Sell'], self.oh[0][0], self.oh[0][1], self.oh[0][2], self.oh[0][3],
                    self.oh[0][4], self.oh[0][5], self.oh[0][6], self.oh[0][7]]
        else:
            return []
        
    def make_choice(self, ch): #0=buy, 1=sell, 2=hold
        if ch == 0:
            if self.settle >= 0:
                self.settle = self.settle-1
                self.reward = self.reward-self.st[-1][1]
                self.choice = [1,0,0]
                self.ch_price = self.st[-1][0]
        elif ch == 1:
            if self.settle <= 0:
                self.settle = self.settle+1
                self.reward = self.reward+self.st[-1][0]
                self.choice = [0,1,0]
                self.ch_price = self.st[-1][1]
        
        tmp = self.reward
        if self.settle == 0:
            self.choice = [0,0,1]
            self.ch_price = 0
            self.reward = 0
            return tmp
        
        return 0    
            
    def get_new_day(self):
        self.day.initialize()
        start = self.day.step()
        print(start)
        self.st = [start]
        self.state = self.init_ohlc
        self.choice = [0,0,1]
        self.ch_price = 0
        self.reward = 0
        self.settle = 0

In [ ]:
s = stock()
s.get_first_ohlc()

t = s.next_data()
boop = [0]
while len(boop) is not 0:
    boop = s.next_data()
    r = s.make_choice(0)
    boop = s.next_data()
    r = r+s.make_choice(1)
    print(r)

This was an enviroment for an A3C algorthm to attempt to learn to buy and sell stocks. This algorithm would injest the data (as csv) clean it and extract the information I wanted from it. It would then transform the data into ohlc, which I would try to predict with a separate rnn. this information was then presented as a state to the A3C algorithm, which would try to guess what the best choice would be.